In [1]:
!pip install torch==1.4.0
#504

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
from collections import  OrderedDict
from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader
import numpy as np
import pickle
import torch
import time
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import pickle
from torch.utils.data import DataLoader
import random
from utils import save_model,test_model,PositoinEncoder,TransformerOptimizer
from dataset import MyDataSet

import logging
logger = logging.getLogger(__name__)                                                                                                                                                                                                          
logger.setLevel(level = logging.INFO)
handler = logging.FileHandler('age_lstm_dfm5.log')
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logger.addHandler(handler)
logger.addHandler(console)


merge_data = pd.read_pickle('prepared_data/merge_data.pkl')
user_max_min_dic = pickle.load(open("prepared_data/user_max_min_dic",'rb'))
user_df = pd.read_csv('prepared_data/user.csv')

use_col = [  'creative_id',  'ad_id', 'product_id',
       'product_category', 'advertiser_id', 'industry']

#加载预处理的embedding layer

    

In [2]:
emblayer_dic = {}
emblayer_dic["user_id"] = pickle.load(open("emb_layer/emb_layer_{}.pkl".format("user_id"),'rb'))
#emblayer_dic["creative_id_dw"] = pickle.load(open("emb_layers/emb_layer_{}.pkl".format("creative_iddw"),'rb'))

for col in use_col:
    emblayer_dic[col] = pickle.load(open("emb_layer/emb_layer_{}.pkl".format(col),'rb'))

In [3]:
train_ids = []
test_ids = []
for i in range(3000000):
    if i%5==0:
        test_ids.append(i+1)
    else:
        train_ids.append(i+1)
train_dst = MyDataSet(train_ids,merge_data ,user_df ,user_max_min_dic,emblayer_dic = emblayer_dic,user_columns =use_col )
train_loader = DataLoader(train_dst,num_workers=6,batch_size=400)
test_dst = MyDataSet(test_ids,merge_data ,user_df ,user_max_min_dic,user_columns =use_col ,emblayer_dic = emblayer_dic,is_train=False)
test_loader = DataLoader(test_dst,num_workers=6,batch_size=400)

In [4]:
class MyModel(nn.Module):
    def __init__(self,modeltype,use_col,dmodel = 512,):
        super(MyModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(dmodel,nhead=8)
        encoder_norm = nn.LayerNorm(dmodel)
        self.encoder = nn.TransformerEncoder(encoder_layer, 1, encoder_norm)
        self.tfmodel = nn.Transformer(d_model=dmodel, nhead=8, num_encoder_layers=4,
                 num_decoder_layers=4, dim_feedforward=dmodel*4, dropout=0.2,
                 activation="relu")
        self.ufc = nn.Linear(256,dmodel)
        #self.user_creative_emb_layer = [emblayer_dic['user_id'].cuda(),None]#emblayer_dic['creative_id_dw'].cuda()]
        #self.emb_layers = [emblayer_dic[key].cuda() for key in use_col]
        self.char_embedding_size = 0
        self.dropout = nn.Dropout(0.2)
        self.char_embedding_size = 16
        #self.emb_lay =  nn.Embedding(92,self.char_embedding_size)
        self.cfc = nn.Linear(400*6+1,dmodel)
        max_ls = [4445720,3812202,44314,18,62965,335]
        #self.emb_layers = [nn.Embedding(key+1,128).cuda() for key in max_ls]
        self.ufc = nn.Linear(912+(400*6 +1)*2 ,dmodel)
        #self.outfc1 = nn.Linear(dmodel+self.char_embedding_size,64)
        self.outfc1 = nn.Linear(dmodel,64)
        #out_dim = 10 if modeltype=='age' else 2
        self.outfc2 = nn.Linear(64,10)
        self.outfc3 = nn.Linear(64,2)
        #self.PE = PositoinEncoder(dmodel=dmodel)
    def get_dim(self,dim):
        return int(12*(1+np.log(dim**0.5)))
    def forward(self,uid,raw_arr,click,time):
        """
        模型输入 user_id  user_id 对应的use_col  click_times time
        """
        # embedding 部分
        uid = uid.squeeze(1)
        #cid = raw_arr
        cid = torch.cat([raw_arr,click.unsqueeze(2)],dim=2)
        #cid :cat 所有的id emb后的结果，一共4*128 + 64*2 +1 (click_times)维
        #uid : deepwalk uid embedding
        # 往下是模型部分
        cid_sum = cid.sum(dim=1)
        cid_max = cid.max(dim=1).values
        uid = torch.cat([uid,cid_sum,cid_max],dim=1)
        mask = cid.sum(dim=2)
        for i,t in enumerate(mask):
            t[:torch.where(t!=0)[0].max()+1] = 1
        mask = (mask-1).bool()
#         size = d_time.size()
#         d_time = self.emb_lay(d_time.long().view(size[1]*size[0])).view(size[0],size[1],-1)
#         cid = torch.cat([cid,d_time],dim=2)
        cid = self.cfc(cid)
        cid = self.encoder(cid)
        cid = cid.transpose(0,1)
        uid = uid.unsqueeze(1)
        uid = self.ufc(uid).transpose(0,1)      
        out = self.tfmodel(cid,uid,src_key_padding_mask=mask)
        out = out.squeeze(0)
        out = self.outfc1(out)
        out = F.relu(out)
        out = self.dropout(out)
        out1 = self.outfc2(out)
        out2 = self.outfc3(out)
        return out1,out2

In [14]:
def train(continue_from,train_loader,test_loader):
    #continue_from = "best_transformer_age5.pth"
    modeltype = "age"
    model = MyModel(modeltype,use_col)
    model = model.cuda()

    #warm up optimizer
    optimizer = TransformerOptimizer(
            torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-08),
            0.3,
            512,
            8000)

    start_epoch = 0
    if os.path.exists(continue_from ):
            naive_model = torch.load(continue_from , map_location=lambda storage, loc:storage)
            new_state_dict = OrderedDict()
            for k, v in naive_model["state"].items():
                    new_state_dict[k] = v 
            model.load_state_dict(new_state_dict)
            start_epoch = naive_model["epoch"]+1
            optimizer.load_state_dict(naive_model["optimizer"])
            optimizer.step_num = naive_model['optimizer_step']
            logger.info("load model finish")
    losses_age = []
    losses_gender= []
    best_acc_age = 0 
    best_acc_gender = 0 
    entroy=nn.CrossEntropyLoss()#mseloss = nn.MSELoss()
    model = nn.DataParallel(model)
    best_age_acc = 0
    best_gender_acc = 0
    for epo in range(start_epoch,20):
        model.train()
        #start = time.time()
        for i,(data) in enumerate(train_loader):
            raw_arr,click,times,label_age,label_gender,user_id,user_id_emb = data
            label_age = label_age.cuda()
            label_gender = label_gender.cuda()
            click = click.cuda()
            times= times.cuda()
            user_id_emb = user_id_emb.cuda()
            raw_arr= raw_arr.cuda()
            embs1,embs2  = model(user_id_emb,raw_arr,click,times)
            loss_age = entroy(embs1,label_age-1)
            loss_gender = entroy(embs2,label_gender-1)
            loss = loss_age+loss_gender
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 400)
            optimizer.step()
            losses_age.append(loss_age.item())
            losses_gender.append(loss_gender.item())
            if (i+1)%2000==0: 
                #print((time.time()-start)/200*len(train_loader))
                for g in optimizer.optimizer.param_groups:
                    logger.info("now learn rate : "+str(g['lr']) )
                mean_loss_age = sum(losses_age)/len(losses_age)
                mean_loss_gender = sum(losses_gender)/len(losses_gender)
                logger.info('cpEpoch:{0} \t step:{1}/{2} \t '
                                    'mean loss:{3} \t{4}'.format(  
                                                            (epo + 1), (i + 1),len(train_loader),mean_loss_age,mean_loss_gender))
        acc_age,acc_gender,loss_age,loss_gender = test_model(model,test_loader,entroy)
        logger.info("acc:{},{} loss:{}{}".format(acc_age,acc_gender,loss_age,loss_gender))
        if acc_age>best_acc_age:
            best_acc_age = acc_age
            save_model(model,epo,optimizer,model_name = continue_from)
        if acc_gender>best_gender_acc:
            best_gender_acc = acc_gender
            save_model(model,epo,optimizer,model_name = continue_from + "gender.pth")

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
modeltype = 'age'
batch_size = 400
skf = StratifiedKFold(n_splits=5, random_state=10086, shuffle=True) 
userid_ls =np.array([i+1 for i in range(3000000)])
id_ls =np.array([0 for i in range(3000000)])
for index, (train_index, test_index) in enumerate(skf.split(userid_ls.reshape(-1,1),id_ls)):
    torch.cuda.empty_cache()
    train_x, test_x = userid_ls[train_index,], userid_ls[test_index,]
    continue_fom = "age_tf_pre_encoder__5fload_{}.pth".format(index)
    train_dst = MyDataSet(train_x,merge_data ,user_df ,user_max_min_dic,emblayer_dic = emblayer_dic,user_columns =use_col )
    train_loader = DataLoader(train_dst,num_workers=8,batch_size=400,shuffle=True)
    test_dst = MyDataSet(test_x,merge_data ,user_df ,user_max_min_dic,user_columns =use_col ,emblayer_dic = emblayer_dic,is_train=False)
    test_loader = DataLoader(test_dst,num_workers=8,batch_size=400)
    
    train(continue_fom,train_loader,test_loader)


load model finish
now learn rate : 6.320623710592894e-05
cpEpoch:8 	 step:2000/6000 	 mean loss:1.225194727599621 	0.1607104297466576
now learn rate : 6.181691938085502e-05
cpEpoch:8 	 step:4000/6000 	 mean loss:1.2247975927591324 	0.16023606195300819
now learn rate : 6.051536478449089e-05
cpEpoch:8 	 step:6000/6000 	 mean loss:1.2240863166848819 	0.1596579452926914


0 1500
300 1500
600 1500
900 1500
1200 1500


acc:0.5041933333333334,0.9477433333333334 loss:1.20141958292325350.15217154809087516
now learn rate : 5.9292706128157114e-05
cpEpoch:9 	 step:2000/6000 	 mean loss:1.221916658371687 	0.1593578636618331
now learn rate : 5.814128183869147e-05
cpEpoch:9 	 step:4000/6000 	 mean loss:1.2208348897337913 	0.15899781892299653
now learn rate : 5.70544330734548e-05
cpEpoch:9 	 step:6000/6000 	 mean loss:1.2202684439917406 	0.15889705659387013


0 1500
300 1500
600 1500
900 1500
1200 1500


acc:0.504185,0.94739 loss:1.20604643694559740.15859783238420883
now learn rate : 5.6026341062549705e-05
cpEpoch:10 	 step:2000/6000 	 mean loss:1.218501488489764 	0.1584865437171289
now learn rate : 5.505189557950345e-05
cpEpoch:10 	 step:4000/6000 	 mean loss:1.2171849153488874 	0.1583700890582986
now learn rate : 5.412658773652741e-05
cpEpoch:10 	 step:6000/6000 	 mean loss:1.2164273548788493 	0.1582292478096982


...

acc:0.50525,0.9476766666666666 loss:1.2064536329905190.15253529386719067
now learn rate : 5.0843076051247535e-05
cpEpoch:12 	 step:2000/6000 	 mean loss:1.211318730546878 	0.15726249388748637
now learn rate : 5.011148285857957e-05
cpEpoch:12 	 step:4000/6000 	 mean loss:1.2101764706543514 	0.15703504726210876
now learn rate : 4.941058844013093e-05
cpEpoch:12 	 step:6000/6000 	 mean loss:1.209301362514496 	0.15686737368181347


0 1500
300 1500
600 1500
900 1500
1200 1500


acc:0.5056266666666667,0.9479116666666667 loss:1.20709637920061750.15176120065152646
now learn rate : 4.873830421031591e-05
cpEpoch:13 	 step:2000/6000 	 mean loss:1.2078748256079852 	0.15662532100011595
now learn rate : 4.80927352539916e-05
cpEpoch:13 	 step:4000/6000 	 mean loss:1.2067795534379342 	0.15639013616894099
now learn rate : 4.747215783204687e-05
cpEpoch:13 	 step:6000/6000 	 mean loss:1.2059038745727804 	0.15623053984240526


0 1500
300 1500
600 1500
900 1500
1200 1500


acc:0.50603,0.947465 loss:1.20388102110226950.15176270091036956
now learn rate : 4.6875e-05
cpEpoch:14 	 step:2000/6000 	 mean loss:1.2046359136575147 	0.15594997382654172


In [12]:
acc_age,acc_gender,loss_age,loss_gender = test_model(model,test_loader,entroy)

0 1500
300 1500
600 1500
900 1500
1200 1500


In [16]:
1

1

In [10]:
modeltype = "age"
model = MyModel(modeltype,use_col)
model = model.cuda()

#warm up optimizer
optimizer = TransformerOptimizer(
        torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-08),
        0.3,
        512,
        8000)

start_epoch = 0
if os.path.exists(continue_from ):
        naive_model = torch.load(continue_from , map_location=lambda storage, loc:storage)
        new_state_dict = OrderedDict()
        for k, v in naive_model["state"].items():
                new_state_dict[k] = v 
        model.load_state_dict(new_state_dict)
        start_epoch = naive_model["epoch"]+1
        optimizer.load_state_dict(naive_model["optimizer"])
        optimizer.step_num = naive_model['optimizer_step']
        logger.info("load model finish")

load model finish


In [9]:
continue_from = 'age_tf_pre_encoder__5fload_0.pth'

In [11]:
entroy=nn.CrossEntropyLoss()